# Arts Council Funding by Org Type

This script combines funding information from the Arts Council Investment Programme, National Lottery Project Grants and Companies House data to explore the relationship between organisation size and the amount of funding received. 

Note: As a proxy for organisation size, we have used the accounting category from Companies House. Accounts categories offer an indication of the size and type of organisation. For definitions of account types, <a href="https://find-and-update.company-information.service.gov.uk/guides/accounts/chooser">see the Companies House definitions</a>.  



Import libraries

In [151]:
import petl as etl

from pipeline_utils.filesystem.paths import RAW_DATA, DATA, SITE

In [152]:
OUT_DIR = SITE / 'insight/funding/_data'
OUT_DIR.mkdir(exist_ok=True, parents=True)

In [153]:
org_data = (
    etl
    .fromcsv(DATA / 'culture_landscape.csv')
    .select(lambda r: r.funding_geo_code == 'E08000021' or r.oslaua == "E08000021")
    .cut('organisation', 'accounts_category', 'company_number')
    .convert('organisation', 'upper')
)

In [154]:
corrections = etl.lookupone(
    etl.fromcsv(RAW_DATA / 'culture-landscape/2-company-corrections.csv'),
    key='organisation', value='match'
)

In [155]:
grants_data = (
    etl
    .fromcsv(RAW_DATA / 'arts-council-project-grants.csv')
    .convert('Recipient', corrections)
    .convertnumbers()
    .rename({ 'Recipient': 'recipient', 'Award amount': 'funding' })
    .cut('recipient', 'funding')
)

In [156]:
invest_data = (
    etl
    .fromcsv(RAW_DATA / 'arts-council-investment-programme.csv')
    .selecteq('Local authority', 'Newcastle upon Tyne')
    .convert('Applicant Name', corrections)
    .convertnumbers()
    .rename({ 'Applicant Name': 'recipient', '2023-26 Annual Funding (Offered 4 Nov 2022)': 'funding' })
    .cut('recipient', 'funding')
)

In [157]:
threesixty_data_by_name, threesixty_data_by_number = (
    etl
    .fromcsv(DATA / 'grants_by_recipient.csv')
    .cutout('total_grants')
    .convert('funding', float)
    .biselect(lambda r: r.company_number == '')
)

Get funding by name. May want to do some fuzzy matching of names to the targets here.

In [158]:
funding_by_name = (
    etl.cat(
        grants_data,
        invest_data,
        threesixty_data_by_name
    )
    .convert('recipient', 'upper')
    .aggregate('recipient', sum, 'funding', field='funding_by_name')
)

In [159]:
funding_by_company_number = (
    etl.cat(
        threesixty_data_by_number
    )
    .aggregate('company_number', sum, 'funding', field='funding_by_company_number')
)

In [160]:
total_funding = (
    org_data
    .leftjoin(funding_by_name, lkey='organisation', rkey='recipient')
    .leftjoin(funding_by_company_number, lkey='company_number', rkey='company_number')
    .convert(('funding_by_name', 'funding_by_company_number'), float)
    .addfield('funding', lambda r: (r.funding_by_name or 0) + (r.funding_by_company_number or 0))
    .aggregate('accounts_category', sum, 'funding', field='total_funding')
    .sort('total_funding', reverse=True)
    .replace('accounts_category', '', 'UNMATCHED')
)

In [161]:
total_funding.tocsv(OUT_DIR / 'grant_by_org_size.csv')

total_funding

accounts_category,total_funding
UNMATCHED,50511761.33
TOTAL EXEMPTION FULL,9152648.68
FULL,7738186.0
GROUP,3985198.0
MICRO ENTITY,1081987.42
